In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv(r'C:\Users\hp\Desktop\heart.csv')

# Split the data into training and testing
X = data.drop(columns=['target'])
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a custom early stoppinng callback that stops training the model after the accuracy is 100%
# The custom callback is from tensorflow.keras.callbacks.Callback
class MyThresholdCallback(keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        accuracy = logs["val_binary_accuracy"]
        if accuracy >= self.threshold:
            self.model.stop_training = True
            
            
early_stopping = MyThresholdCallback(threshold=1.0)

In [7]:
# Create the model with 4 dense layers (dense layers contain our neurons)
# The model uses BatchNormilzation() in order to scale our data in the range: [0, 1]
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', input_shape=[X.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

#  I Compiled the model using adam optimizer
# I Measured the loss with binary_crossentropy
# Set metrics to binary_accuracy, which lets us know how accurate the model is
# The accuracy lies on a scale of [0.0, 1.0],
# In order to convert this accuracy to percentage, move the decimal point to the right twice (acc*100)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Finally fit the model, training it with X_train and y_train, and validating it with X_test and y_test
# Use 25 data points per epoch (the amount of epochs is the amount of times we feed the model data)
# Use the custom callback defined earlier that stops the model at 100% accuracy
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=25,
    epochs=30,
    callbacks=[early_stopping]
)


Epoch 1/30
33/33 [==============================] - 15s 84ms/step - loss: 0.4673 - binary_accuracy: 0.8207 - val_loss: 0.5560 - val_binary_accuracy: 0.7854
Epoch 2/30
33/33 [==============================] - 2s 52ms/step - loss: 0.1918 - binary_accuracy: 0.9280 - val_loss: 0.7661 - val_binary_accuracy: 0.5122
Epoch 3/30
33/33 [==============================] - 1s 44ms/step - loss: 0.1734 - binary_accuracy: 0.9329 - val_loss: 0.5418 - val_binary_accuracy: 0.7610
Epoch 4/30
33/33 [==============================] - 1s 45ms/step - loss: 0.1065 - binary_accuracy: 0.9524 - val_loss: 0.4890 - val_binary_accuracy: 0.7122
Epoch 5/30
33/33 [==============================] - 1s 42ms/step - loss: 0.0901 - binary_accuracy: 0.9646 - val_loss: 0.3949 - val_binary_accuracy: 0.8341
Epoch 6/30
33/33 [==============================] - 1s 43ms/step - loss: 0.0924 - binary_accuracy: 0.9683 - val_loss: 0.3816 - val_binary_accuracy: 0.8390
Epoch 7/30
33/33 [==============================] - 2s 46ms/step - lo

In [18]:
# Now plot our increase/decrease in percentage and loss
# The left graph represents percent, and the right graph represents loss
# i'm looking for a high percent and a low loss
#history_df = pd.DataFrame(history.history)

f, (ax1, ax2) = plt.subplots(1, 2)
#f.figure.set_figheight(5)
#f.figure.set_figwidth(15)

ax1.plot(history_df.index, history_df['binary_accuracy'])
ax1.plot(history_df.index, history_df['val_binary_accuracy'])

ax2.plot(history_df.index, history_df['loss'])
ax2.plot(history_df.index, history_df['val_loss'])